# 한국어 혐오 발언 다중 레이블 분류

In [1]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

# 데이터셋 로드 및 확인

In [2]:
from datasets import load_dataset

dataset = load_dataset('jeanlee/kmhas_korean_hate_speech')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

kmhas_korean_hate_speech.py:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/5.24M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/579k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78977 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21939 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 78977
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 8776
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21939
    })
})

- 데이터셋의 [깃허브](https://github.com/adlnlp/K-MHaS)로부터 확인할 수 있는 각 레이블이 의미

      class_label:
        names:
          0: origin (출신차별)
          1: physical (외모차별)
          2: politics (정치성향차별)
          3: profanity (혐오욕설)
          4: age (연령차별)
          5: gender (성차별)
          6: race (인종차별)
          7: religion (종교차별)
          8: not_hate_speech (혐오아님)

## 전처리

In [4]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm import tqdm

import csv
import os

import torch

# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

In [5]:
train = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")
validation = load_dataset("jeanlee/kmhas_korean_hate_speech", split="validation")
test = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [6]:
train_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', train['text']))
valid_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', validation['text']))
test_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', test['text']))

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

In [8]:
def multi_label(example):
  mlb_label = mlb.fit_transform(example['label'])
  float_arr = np.vstack(mlb_label[:]).astype(float)
  update_label = float_arr.tolist()

  return update_label

In [9]:
train_labels = multi_label(train)
validation_labels = multi_label(validation)
test_labels = multi_label(test)

In [10]:
test_sentences[:5]

['[CLS]그만큼 길예르모가 잘했다고 보면되겠지 기대되네 셰이프 오브 워터[SEP]',
 '[CLS]"1. 8넘의 문재앙"[SEP]',
 '[CLS]"문재인 정권의 내로남불은 타의 추종을 불허하네. 자한당 욕할거리도 없음."[SEP]',
 '[CLS]"짱개들 지나간 곳은 폐허된다 ㅋㅋ"[SEP]',
 '[CLS]곱창은 자갈치~~~~~[SEP]']

In [11]:
test_labels[:5]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]

## 전처리

In [12]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [13]:
MAX_LEN = 128

def data_to_tensor(sentences, labels):
  tokenizer_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenizer_texts]

  input_ids = [torch.tensor(ids) for ids in input_ids]
  input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)

  attention_masks=[]
  for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)

  tensor_input= torch.tensor(input_ids)
  tensor_label = torch.tensor(labels)
  tensor_mask = torch.tensor(attention_masks)

  return tensor_input, tensor_label, tensor_mask

In [14]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs, validation_labels, validation_masks = data_to_tensor(valid_sentences, validation_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

<ipython-input-13-2d58f1b825ec>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_input= torch.tensor(input_ids)


In [15]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [16]:
print('훈련 데이터의 크기:', len(train_labels))
print('검증 데이터의 크기:', len(validation_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 78977
검증 데이터의 크기: 8776
테스트 데이터의 크기: 21939


## GPU 셋팅

In [17]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## 모델 로드

In [18]:
num_labels = 9

model = BertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=num_labels,problem_type='multi_label_classification')
model.cuda()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [19]:
optimizer = AdamW(model.parameters(),
                 lr = 2e-5,
                 eps=1e-8)

In [25]:
epochs=2
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

In [26]:
def format_time(elapsed):
  elapsed_rounded = int(round(elapsed))
  return str(datetime.timedelta(seconds=elapsed_rounded))

In [35]:
def multi_label_metrics(predictions, labels, threshold=0.5):
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions))

  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs >= threshold)] = 1

  y_true = labels

  accuracy = accuracy_score(y_true, y_pred)
  f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
  f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
  f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
  roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')

  metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average,
               'roc_auc': roc_auc}

  return metrics

In [37]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))  # 확률로 변환

    # 예측값을 0 또는 1로 설정
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # y_true와 y_pred의 형식이 동일해야 함 (다중 라벨 인디케이터 형식)
    y_true = labels

    # accuracy 계산 (평균 정확도)
    accuracy = accuracy_score(y_true, y_pred)

    # F1 score 계산 (macro, micro, weighted)
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)

    # ROC AUC score 계산
    roc_auc = roc_auc_score(y_true, y_pred, average='micro')

    # 계산된 메트릭들을 딕셔너리로 반환
    metrics = {
        'accuracy': accuracy,
        'f1_macro': f1_macro_average,
        'f1_micro': f1_micro_average,
        'f1_weighted': f1_weighted_average,
        'roc_auc': roc_auc
    }

    return metrics


## 모델 학습

In [28]:
seed_val = 810
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [29]:
model.zero_grad()
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    t0 = time.time()
    total_loss = 0

    model.train()

    for step,batch in tqdm(enumerate(train_dataloader)):
      if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch

      outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

      loss = outputs[0]
      total_loss += loss.item()
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      scheduler.step()

      model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

======== Epoch 1 / 2 ========


500it [10:00,  1.20s/it]

  Batch   500  of  2,469.    Elapsed: 0:10:00.


1000it [20:00,  1.20s/it]

  Batch 1,000  of  2,469.    Elapsed: 0:20:01.


...

  Batch 1,500  of  2,469.    Elapsed: 0:30:04.


2000it [40:04,  1.20s/it]

  Batch 2,000  of  2,469.    Elapsed: 0:40:05.


2469it [49:27,  1.20s/it]


  Average training loss: 0.0529
  Training epcoh took: 0:49:28


## 검증데이터 평가

In [43]:
t0 = time.time()

model.eval()

accum_logits, accum_label_ids = [], []

for batch in validation_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
    outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().to('cpu').numpy()
    labels_ids = b_labels.to('cpu').numpy()

    accum_logits.extend(logits)
    accum_label_ids.extend(labels_ids)

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = multi_label_metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))

[[-7.172347  -7.048173  -7.034955  ... -7.918266  -8.164259   5.824846 ]
 [-7.923737  -5.380665  -7.456469  ... -8.177288  -8.985796   4.673845 ]
 [-5.211823   4.0031314 -5.076598  ... -5.643192  -5.6834383 -4.5146813]
 ...
 [ 2.300257  -3.7418969 -5.3544397 ... -3.4612746 -5.9489164 -3.4212723]
 [-4.5295615  4.4461346 -4.472451  ... -5.1806884 -6.297585  -3.8922553]
 [-7.356191  -7.1069937 -7.160541  ... -8.069308  -8.184309   5.7421126]]
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
Accuracy: 0.8052
F1 (Macro) Score: 0.7568
F1 (Micro) Score: 0.8522
F1 (Weighted) Score: 0.8517
ROC-AUC: 0.9135


## 모델저장 로드

In [44]:
%pwd

'/content'

In [45]:
%mkdir model

In [46]:
path = '/content/model/'

In [47]:
torch.save(model.state_dict(), path+"BERT_multilabel_model.pt")

In [48]:
model.load_state_dict(torch.load(path+"BERT_multilabel_model.pt"))

<All keys matched successfully>

## 테스트데이터 평가

In [50]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for step, batch in tqdm(enumerate(test_dataloader)):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()

    accum_logits.extend(logits)
    accum_label_ids.extend(label_ids)

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = multi_label_metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))
print("ROC-AUC: {0:.4f}".format(results['roc_auc']))

100it [00:34,  2.84it/s]

  Batch   100  of    686.    Elapsed: 0:00:34.


200it [01:08,  3.02it/s]

  Batch   200  of    686.    Elapsed: 0:01:08.


300it [01:41,  2.99it/s]

...

  Batch   500  of    686.    Elapsed: 0:02:49.


600it [03:21,  2.98it/s]

  Batch   600  of    686.    Elapsed: 0:03:22.


686it [03:50,  2.97it/s]


Accuracy: 0.8007
F1 (Macro) Score: 0.7679
F1 (Micro) Score: 0.8513
F1 (Weighted) Score: 0.8512
ROC-AUC: 0.9129


## 예측

In [51]:
from transformers import pipeline

In [52]:
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512,
               top_k=None, function_to_apply='sigmoid')

Device set to use cuda:0


In [53]:
result = pipe('틀니들은 왜 그렇게 민폐를 끼치냐?')
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_4', 'score': 0.9899629950523376}, {'label': 'LABEL_2', 'score': 0.025329608470201492}, {'label': 'LABEL_8', 'score': 0.0066582090221345425}, {'label': 'LABEL_5', 'score': 0.0066312928684055805}, {'label': 'LABEL_1', 'score': 0.005534169729799032}, {'label': 'LABEL_0', 'score': 0.005304795224219561}, {'label': 'LABEL_3', 'score': 0.00507295411080122}, {'label': 'LABEL_7', 'score': 0.0029795293230563402}, {'label': 'LABEL_6', 'score': 0.0012406264431774616}]]


In [54]:
label_dict = {'LABEL_0' : '출신차별', 'LABEL_1' : '외모차별', 'LABEL_2' : '정치성향차별', \
              'LABEL_3': '혐오욕설', 'LABEL_4': '연령차별', 'LABEL_5': '성차별', 'LABEL_6' : '인종차별', \
              'LABEL_7': '종교차별', 'LABEL_8': '해당사항없음'}

In [55]:
def prediction(text):
  result = pipe(text)
  return [label_dict[res['label']] for res in result[0] if res['score'] > 0.5]

In [64]:
prediction('틀니들은 왜 그렇게 민폐를 끼치냐?')

['연령차별']